# Setting Up Data

In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [2]:
# Load in data
data = pd.read_csv('Resources/Data.csv')
data.head()

,address,city,zip,utility_provider,average_bill,number_of_panels,panel_model,principal,available_taxcredit,after_taxcredit,monthly_rate,loan_term(months),montly_payment,if_cash,kW,add_battery,net_with_battery,monthly_battery
0,713 East 9th Street,San Juan,78589,AEP,199.92,34,SunPower,34639.2,9006.19,25633.01,0.0004,240,112.03,0.0,12.24,48639.2,35993.01,157.31
1,705 Dellwood Drive,Laredo,78045,AEP,217.56,37,SunPower,37695.6,9800.86,27894.74,0.0004,240,121.92,0.0,13.32,51695.6,38254.74,167.20
2,4310 Boots Drive,Killeen,76549,Oncor,211.68,36,SunPower,36676.8,9535.97,27140.83,0.0004,240,118.62,0.0,12.96,50676.8,37500.83,163.90
3,3307 Begay,Laredo,78046,AEP,323.40,55,SunPower,56034.0,14568.84,41465.16,0.0004,240,181.23,0.0,19.80,70034.0,51825.16,226.51
4,11112 Entrada Loop,Laredo,78045,AEP,235.20,40,SunPower,40752.0,10595.52,30156.48,0.0004,240,131.80,0.0,14.40,54752.0,40516.48,177.09


In [3]:
# Get the necessary data to determine the number of panels needed
necessary_data = data.drop(['address', 'if_cash', 'kW', 'add_battery', 'net_with_battery', 'monthly_battery'],
                          axis = 1)
necessary_data.head()

,city,zip,utility_provider,average_bill,number_of_panels,panel_model,principal,available_taxcredit,after_taxcredit,monthly_rate,loan_term(months),montly_payment
0,San Juan,78589,AEP,199.92,34,SunPower,34639.2,9006.19,25633.01,0.0004,240,112.03
1,Laredo,78045,AEP,217.56,37,SunPower,37695.6,9800.86,27894.74,0.0004,240,121.92
2,Killeen,76549,Oncor,211.68,36,SunPower,36676.8,9535.97,27140.83,0.0004,240,118.62
3,Laredo,78046,AEP,323.40,55,SunPower,56034.0,14568.84,41465.16,0.0004,240,181.23
4,Laredo,78045,AEP,235.20,40,SunPower,40752.0,10595.52,30156.48,0.0004,240,131.80


In [4]:
# Get the max average bill
necessary_data['average_bill'].max()

364.56

In [5]:
# Use the get_dummies method on non float columns City, Electric Utility, and panel_model
#clean_data = pd.get_dummies(necessary_data, columns=['city', 'utility_provider', 'panel_model'])
#clean_data.head()

# Begin Multiple Linear Regression Model

In [6]:
# Set the target and features
y = necessary_data['number_of_panels']
X = necessary_data['average_bill']

In [7]:
# Splitting into Train and Test sets into an 80/20 split.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=85)

X_train.shape

(278,)

In [8]:
# Training the data using linear regression
model = LinearRegression()
model.fit(X_train.values.reshape(-1,1), y_train)

LinearRegression()

In [9]:
# Make predictions
y_prediction = model.predict(X_test.values.reshape(-1,1))
y_prediction

array([43., 30., 30., 40., 41., 19., 20., 38., 36., 46., 45., 57., 55.,
       60., 34., 30., 28., 32., 30., 44., 52., 35., 30., 32., 41., 47.,
       48., 37., 34., 22., 46., 41., 41., 40., 20., 31., 53., 30., 52.,
       24., 57., 35., 41., 32., 38., 30., 43., 56., 49., 22., 42., 35.,
       28., 27., 30., 44., 42., 30., 34., 26., 36., 38., 40., 37., 30.,
       23., 30., 36., 33., 32.])

In [10]:
# importing r2_score module
# Code received from https://www.analyticsvidhya.com/blog/2021/05/multiple-linear-regression-using-python-and-scikit-learn/
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print('r2_score is',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_prediction))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2_score is 1.0
mean_sqrd_error is== 6.851820433919645e-30
root_mean_squared error of is== 2.617598218581233e-15


# Test

In [11]:
# Test new inputs
new_bill = [[250]]

# Predict using the new average bill
new_panels_needed = model.predict(new_bill).round()

# Determine total cost
TC_try = new_panels_needed*360*2.83
TC = TC_try.round()
# Determine Net Cost
TNC_try = TC*0.74
TNC = TNC_try.round()
# Determine monthly payment 20 Year
MP_20_try = TNC*0.00044029517 / 0.10073793525
MP_20 = MP_20_try.round()
# Determine monthly payment 12 year
MP_12_try = TNC/144
MP_12 = MP_12_try.round()
# Determine Yearly Savings
monthly_savings = new_bill - MP_20
Yearly_savings = monthly_savings * 12

# Print information for client
print(f'Panels: {new_panels_needed}\nTotal Cost: {TC} \n20 Year Monthly Payment: {MP_20}\n12 Year Monthly Payment: {MP_12}\nYearly Savings: {Yearly_savings}')

Panels: [43.]
Total Cost: [43808.] 
20 Year Monthly Payment: [142.]
12 Year Monthly Payment: [225.]
Yearly Savings: [[1296.]]
